# Create an exclusion plot using fits distributed on GCP

In [ ]:
import asyncio
import glob
import json
import os
import random
import re
import shlex
import subprocess
import sys
import time
from pathlib import Path

import aiohttp
import matplotlib.pyplot as plt
import numpy as np

%matplotlib notebook

## Whole lot of helper and plotting functions we're going to ignore for now

In [ ]:
def make_harvest_from_result(result, masses):
    return {
        "CLs": result["CLs_obs"],
        "CLsexp": result["CLs_exp"][2],
        "clsd1s": result["CLs_exp"][1],
        "clsd2s": result["CLs_exp"][0],
        "clsu1s": result["CLs_exp"][3],
        "clsu2s": result["CLs_exp"][4],
        "covqual": 3,
        "dodgycov": 0,
        "excludedXsec": -999007,
        "expectedUpperLimit": -1,
        "expectedUpperLimitMinus1Sig": -1,
        "expectedUpperLimitMinus2Sig": -1,
        "expectedUpperLimitPlus1Sig": -1,
        "expectedUpperLimitPlus2Sig": -1,
        "fID": -1,
        "failedcov": 0,
        "failedfit": 0,
        "failedp0": 0,
        "failedstatus": 0,
        "fitstatus": 0,
        "mn1": masses[2],
        "mn2": masses[1],
        "mode": -1,
        "msb": masses[0],
        "nexp": -1,
        "nofit": 0,
        "p0": 0,
        "p0d1s": -1,
        "p0d2s": -1,
        "p0exp": -1,
        "p0u1s": -1,
        "p0u2s": -1,
        "p1": 0,
        "seed": 0,
        "sigma0": -1,
        "sigma1": -1,
        "upperLimit": -1,
        "upperLimitEstimatedError": -1,
        "xsec": -999007,
    }


def harvest_results(regions):
    pattern = re.compile(r"sbottom_(\d+)_(\d+)_(\d+)")

    dataList = []
    for region in regions:
        harvest = []
        files = "results/region{region}.result.sbottom_*_*_*.json".format(
            region=region,
        )
        for fname in glob.glob(files):
            result = json.load(open(fname))
            m = pattern.search(fname)
            masses = list(map(int, m.groups()))
            # only use 60 GeV
            if masses[2] != 60:
                continue
            harvest.append(make_harvest_from_result(result, masses))
        dataList.append((f"region{region}", harvest))
    return dataList

In [ ]:
def make_plot(ax, **kwargs):
    ax.cla()
    ax.set_xlim(300, 1700)
    ax.set_ylim(198, 1700)
    r, x = make_results()
    if r is None:
        return

    if x[0][1] and kwargs.get("showPoints", False):
        y = np.asarray([[xx["msb"], xx["mn2"]] for xx in x[0][1]])
        ax.scatter(y[:, 0], y[:, 1], s=20, alpha=0.2)
    if x[1][1] and kwargs.get("showPoints", False):
        y = np.asarray([[xx["msb"], xx["mn2"]] for xx in x[1][1]])
        ax.scatter(y[:, 0], y[:, 1], s=10, alpha=0.2)

    x = np.asarray(array.array("d", r["Band_1s_0"].GetX()))
    y = np.asarray(array.array("d", r["Band_1s_0"].GetY()))

    explabel = r"Expected Limit ($\pm1\sigma$)"
    p = ax.add_patch(
        PolygonPatch(
            Polygon(np.stack([x, y]).T),
            alpha=0.5,
            facecolor=kwargs.get("color", "steelblue"),
            label=explabel,
        ),
    )

    x = np.asarray(array.array("d", r["Exp_0"].GetX()))
    y = np.asarray(array.array("d", r["Exp_0"].GetY()))
    ax.plot(x, y, color="k", linestyle="dashed", alpha=0.5)

    x = np.asarray(array.array("d", r["Obs_0"].GetX()))
    y = np.asarray(array.array("d", r["Obs_0"].GetY()))
    ax.plot(
        x,
        y,
        color="maroon",
        linewidth=2,
        linestyle="solid",
        alpha=0.5,
        label="Observed Limit",
    )
    apply_decorations(ax, kwargs["label"])


def apply_decorations(ax, label):
    ax.set_xlim(300, 1700)
    ax.set_ylim(200, 1700)
    # dictionaries to hold the styles for re-use
    text_fd = dict(ha="left", va="center")
    atlas_fd = dict(weight="bold", style="italic", size=24, **text_fd)
    internal_fd = dict(size=24, **text_fd)

    # actually drawing the text
    ax.text(0.05, 0.9, "ATLAS", fontdict=atlas_fd, transform=ax.transAxes)
    ax.text(0.23, 0.9, label, fontdict=internal_fd, transform=ax.transAxes)

    ax.text(
        0.05,
        0.8,
        "$\\sqrt{s} = 13\\ \\mathrm{TeV}, 139\\ \\mathrm{fb}^{-1}$\n All limits at 95% CL",
        fontdict=text_fd,
        transform=ax.transAxes,
    )
    ax.text(
        0.0,
        1.035,
        r"$\tilde{b}_1\tilde{b}_1$ production ; $\tilde{b}_1\to b \tilde{\chi}_2^0$; $m(\tilde{\chi}_1^0)$ = 60 GeV",
        fontdict=text_fd,
        transform=ax.transAxes,
    )

    ax.text(
        350,
        750,
        r"Kinematically Forbidden $m(\tilde{\chi}_2^0)>m(\tilde{b}_1)$",
        rotation=35.0,
        fontdict=dict(ha="left", va="center", size=15, color="grey"),
    )
    ax.set_xlabel(
        r"$m(\tilde{b}_1)$ [GeV]", fontdict=dict(ha="right", va="center", size=20)
    )
    ax.set_ylabel(
        r"$m(\tilde{\chi}_2^0)$ [GeV]", fontdict=dict(ha="right", va="center", size=20)
    )

    ax.legend(loc=(0.05, 0.6))
    ax.xaxis.set_label_coords(1.0, -0.1)
    ax.yaxis.set_label_coords(-0.15, 1.0)
    ax.plot([200, 1400], [200, 1400], color="grey", linestyle="dashdot")

In [ ]:
data_live = None
done_live = None


def reset_data_live():
    global data_live
    global done_live
    data_live = {"A": {}, "C": {}}
    done_live = {"A": False, "C": False}

In [ ]:
async def from_online(key, session, url, patch):
    region = url[-1]
    resp = await session.post(url, data=patch)
    result = (resp.status, await resp.json())
    data_live[region][key] = result
    return result

In [ ]:
async def fetch_html(url: str, session: aiohttp.ClientSession, patch) -> tuple:
    key, patch = patch
    for i in range(10):
        try:
            result = (key, await from_online(key, session, url, patch))
            return result
        except:
            pass
    return ("404", (key, {}))

In [ ]:
async def make_requests(url, patches: set, **kwargs) -> None:
    async with aiohttp.ClientSession(
        headers={"Content-Type": "application/json"}
    ) as session:
        tasks = []
        for patch in patches:
            tasks.append(
                fetch_html(
                    url=url,
                    session=session,
                    patch=patch,
                )
            )

        results = await asyncio.gather(*tasks)
    return results

In [ ]:
async def run_region(region="A"):
    start = time.time()
    patches = list(
        {
            x: open(x).read()
            for x in glob.glob(f"Region{region}/patch.sbottom_*_60.json")
        }.items()
    )
    print(f"Number of Patches: {len(patches)}")
    results = await make_requests(f"{fitting_service_url}/region{region}", patches[:])
    all_ok = np.all([x[1][0] == 200 for x in results])
    print(f"->Number of Results: {len(results)} {time.time()-start} | {all_ok}")
    done_live[region] = True
    return results

In [ ]:
def plot(ax, data, color="steelblue", label="(in progress)", showInterPolated=True):
    ax.cla()

    hdata = [
        (
            "regionA",
            [make_harvest_from_result(k, r[1]) for k, r in data_live["A"].items()],
        ),
        (
            "regionC",
            [make_harvest_from_result(k, r[1]) for k, r in data_live["C"].items()],
        ),
    ]

    if len(hdata[0][1]) > 3 and len(hdata[1][1]) > 3:
        make_plot(
            ax,
            dataList=hdata,
            label=f"Open Likelihood {label}",
            color=color,
            showPoints=True,
            showInterPolated=showInterPolated,
        )
    else:
        apply_decorations(ax, label=f"Open Likelihood {label}")
    ax.text(
        0.5,
        0,
        f'{len(data_live["A"])+len(data_live["C"])}/{len(data_live["A"])},{len(data_live["C"])}',
    )
    fig.set_tight_layout(True)
    fig.canvas.draw()

In [ ]:
async def plotting_loop():
    global data_live
    global done_live
    while not (done_live["A"] and done_live["C"]):
        plot(ax, data_live, showInterPolated=True)
        await asyncio.sleep(1.0)
    plot(ax, data_live, color="gold", label="")
    print("done plotting")

Setup the figure and canvas so it is ready for updating the plot

In [ ]:
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(9.33, 7)
apply_decorations(ax, label="Open Likelihood (in progress)")
fig.set_tight_layout(True)

## Performing the Fit

In [ ]:
# cloud resources to re-perform stat. analysis
with open("fitting_service_url.txt") as gcp_url_file:
    pyhf_endpoint = str(gcp_url_file.read().rstrip())

In [ ]:
# source probability models needed to reproduce results
! pyhf contrib download "https://www.hepdata.net/record/resource/997020?view=true"

In [ ]:
reset_data_live()
plot(ax, data_live)
start = time.time()
results = await asyncio.gather(run_region("A"), run_region("C"), plotting_loop())
time.time() - start

In [ ]:
fig.savefig("public_likelihood.png")